In [1]:
import folium
import folium.plugins as fp
import json
import pandas as pd

In [2]:
df_stop = pd.read_excel('DS/data/공공자전거 대여소 정보.xlsx')
df_stop

,구분,대여소번호,대여소명,위도,경도,거치대수
0,마포구,101,101. (구)합정동 주민센터,37.549561,126.905754,5
1,마포구,102,102. 망원역 1번출구 앞,37.556000,126.910454,20
2,마포구,103,103. 망원역 2번출구 앞,37.554951,126.910835,14
3,마포구,104,104. 합정역 1번출구 앞,37.550629,126.914986,13
4,마포구,105,105. 합정역 5번출구 앞,37.550007,126.914825,5
...,...,...,...,...,...,...
1532,광진구,3537,아차산 휴먼시아 아파트 옆,37.548489,127.093758,15
1533,성동구,3538,서울숲 IT캐슬,37.550892,127.044762,14
1534,강동구,3539,서원마을,37.563820,127.132843,10
1535,광진구,3541,커먼그라운드,37.545040,127.089958,17


In [3]:
df_stop['구분'].unique()

array(['마포구', '서대문구', '영등포구', '중구', '종로구', '광진구', '성동구', '동대문구', '양천구',
       '용산구', '은평구', '강동구', '강서구', '송파구', '성북구', '중랑구', '강북구', '노원구',
       '도봉구', '금천구', '구로구', '동작구', '관악구', '서초구', '강남구'], dtype=object)

In [4]:
# 구별 대여소의 거치대 수 총합을 데이터프레임으로 생성
df_stop_total = df_stop.groupby('구분')['거치대수'].sum().to_frame()
df_stop_total.head()

,거치대수
구분,
강남구,1210
강동구,773
강북구,451
강서구,1014
관악구,652


In [5]:
map = folium.Map(location=[37.5202, 126.975], zoom_start=10.5)
mc = fp.MarkerCluster(control=False) # Coltrol = Whether the Layer will be included in LayerControls.

map.add_child(mc)
geo_path = 'DS/data/02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

map.choropleth(
    geo_data = geo_str,
    data = df_stop_total,
    columns = [df_stop_total.index, '거치대수'],
    fill_color = 'YlGnBu',
    key_on = 'feature.id'
)

for x in df_stop.index:
    if pd.notnull(df_stop['위도'][x]):
        folium.Marker([df_stop['위도'][x], df_stop['경도'][x]],
                    icon = folium.Icon(color='blue', icon='ok'),
                    popup = '%s<br>%d' %(df_stop['대여소명'][x], df_stop['거치대수'][x])).add_to(mc)
        
map


c:\Users\sesac\.conda\envs\sesac\lib\site-packages\folium\folium.py:413: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [6]:
map.save('folium_test.html')